In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

#sumber:
#http://www.red3d.com/cwr/boids/
#http://www.kfish.org/boids/pseudocode.html

howmanybirds = 100
howmanypredators = 10
iterations = 480

class thing:
    def __init__(self, index):
        self.index  = index
        self.position    = np.random.uniform(0,100,3)
        self.velocity    = np.zeros(3)

    def limitvelocity(self,maxvel):
        if np.linalg.norm(self.velocity) > maxvel:
            self.velocity = (self.velocity/np.linalg.norm(self.velocity))*maxvel

def normalize(vector):
    vector = vector/np.linalg.norm(vector) 
    return vector

def avgflockpos(flock):
    avgfpos = np.zeros(3)
    for i in range(3):
        for thing in flock:
            avgfpos[i] = avgfpos[i] + thing.position[i]
        avgfpos[i] = avgfpos[i]/len(flock)
    return avgfpos

def avgflockvel(flock):
    avgfvel = np.zeros(3)
    for i in range(3):
        for thing in flock:
            avgfvel[i] = avgfvel[i] + thing.velocity[i]
        avgfvel[i] = avgfvel[i]/len(flock)
    return avgfvel

def makeplot():
    global ax
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_xlim3d([0.0, 100.0])
    ax.set_ylim3d([0.0, 100.0])
    ax.set_zlim3d([0.0, 100.0])
    ax.w_xaxis.set_pane_color((175/255,238/255,238/255,1))
    ax.w_yaxis.set_pane_color((175/255,238/255,238/255,1))
    ax.w_zaxis.set_pane_color((175/255,238/255,238/255,1))
    ax.set_facecolor((175/255,238/255,238/255,1))
    ax.view_init(elev=10, azim=0)
    
def initializeplot():
    makeplot()
    avgfpos = avgflockpos(flock)
    birdpos = np.zeros((howmanybirds, 3))
    predpos = np.zeros((howmanypredators, 3))

    for index,thing in enumerate(flock):
        birdpos[index]= thing.position
    bx = birdpos[:,0]
    by = birdpos[:,1]
    bz = birdpos[:,2]

    for index,predator in enumerate(predflock):
        predpos[index]= predator.position
    px = predpos[:,0]
    py = predpos[:,1]
    pz = predpos[:,2]

def movepositions(iteration):
    if howmanypredators > 0:
        for predator in predflock:
            separation = np.zeros(3)
            for anotherpred in predflock:
                difference  = anotherpred.position - predator.position
                distance    = np.linalg.norm(difference)
                if distance < 10 and anotherpred != predator:
                    separation = separation - normalize(difference)/distance
                    
            huntforbirds = np.zeros(3)
            for thing in flock:
                difference  = predator.position - thing.position
                distance    = np.linalg.norm(difference)
                if distance < 10:
                    huntforbirds = huntforbirds - normalize(difference)
                
            huntforbirds = huntforbirds*2

            movepredators = np.zeros(3)
            #https://en.wikipedia.org/wiki/Figure-eight_knot_(mathematics)
            t = (iteration/float(iterations))*4*math.pi + predator.index*(math.pi/4.0)
            movepredators[0] = (2.0+math.cos(2.0*t))*math.cos(3.0*t)
            movepredators[1] = (2.0+math.cos(2.0*t))*math.sin(3.0*t)
            movepredators[2] = math.sin(4*t)
            movepredators = movepredators * 2

            predator.velocity = separation + huntforbirds + movepredators
            predator.limitvelocity(1)
            predator.position = predator.position + predator.velocity

    for thing in flock:
        cohesion = np.zeros(3)
        cohesion = (avgflockpos(flock) - thing.position)*0.03

        separation = np.zeros(3)
        for anotherthing in flock:
            difference  = anotherthing.position - thing.position
            distance    = np.linalg.norm(difference)
            if distance < 10 and anotherthing != thing:
                separation = separation - normalize(difference)/distance

        alignment = np.zeros(3)
        alignment = (avgflockvel(flock) - thing.velocity)*0.125

        plotcenter = np.ones(3)*50
        gotocenter = (plotcenter - thing.position)*0.02

        avoidpredator = np.zeros(3)
        if howmanypredators > 0:
            for predator in predflock:
                difference  = predator.position - thing.position
                distance    = np.linalg.norm(difference)
                if distance < 20:
                    avoidpredator = (avoidpredator - difference)

        thing.velocity = cohesion + separation + alignment + gotocenter + avoidpredator
        thing.limitvelocity(1)
        thing.position = thing.position + thing.velocity

def updateplot(iteration):
    fig.clf()
    makeplot()
    movepositions(iteration)
    birdpos = np.zeros((howmanybirds, 3))
    predpos = np.zeros((howmanypredators, 3))
    
    for index,thing in enumerate(flock):
        birdpos[index]= thing.position
    bx = birdpos[:,0]
    by = birdpos[:,1]
    bz = birdpos[:,2]
    ax.scatter(bx, by, bz, s=90, color='black', marker=r'$\bowtie$')

    if howmanypredators > 0:
        for index,predator in enumerate(predflock):
            predpos[index]= predator.position
        px = predpos[:,0]
        py = predpos[:,1]
        pz = predpos[:,2]
        ax.scatter(px, py, pz, s=100, color='red', marker=r'$\bowtie$')
    ax.view_init(elev=10., azim=iteration)

flock           = [thing(count) for count in range(howmanybirds)]
if howmanypredators > 0:
    predflock   = [thing(count) for count in range(howmanypredators)]
    for predator in predflock:
        predator.position = np.random.uniform(35,65,3)
else:
    predflock   = []

fig = plt.figure()
makeplot()
"""
for i in range(iterations):
updateplot(i)
#plt.title('{0:03d}'.format(i))
filename = 'img{0:03d}.png'.format(i)
plt.savefig(filename, bbox_inches='tight')

#plt.close(fig)
"""
anim = animation.FuncAnimation(fig, updateplot, init_func=initializeplot, frames=iterations, interval=50)
anim.save('video.mp4', fps=24, codec='mpeg4')
HTML(anim.to_html5_video())